# Time Travelling

Another advantage of Iceberg's metadata structure is that it gives us Time Travel for free. Since all we're doing is storing snapshots and moving pointers, time travelling is essentially just asking to see the data at a previous pointer.

In [1]:
import sqlalchemy as sa
import polars as pl
from pyiceberg.catalog.rest import RestCatalog
from IPython.display import display
pl.Config.set_thousands_separator(',')

polars.config.Config

In [2]:
engine = sa.create_engine("trino://trino:@trino:8080/lakekeeper")
catalog = RestCatalog(
    "lakekeeper", uri="http://lakekeeper:8181/catalog", warehouse="lakehouse"
)
house_prices_t = catalog.load_table("housing.staging_prices")

## Python API vs SQL
Pyiceberg offers us some APIs that let us inspect the table metadata - it's all Pyarrow under the hood in Pyiceberg, so we can use polars to pretty-print the dataframes

In [3]:
with pl.Config(thousands_separator=None):
    display(pl.from_arrow(house_prices_t.inspect.history()))

shape: (4, 4)
┌─────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ made_current_at         ┆ snapshot_id         ┆ parent_id           ┆ is_current_ancestor │
│ ---                     ┆ ---                 ┆ ---                 ┆ ---                 │
│ datetime[ms]            ┆ i64                 ┆ i64                 ┆ bool                │
╞═════════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╡
│ 2025-06-06 08:18:03.013 ┆ 376089619872553315  ┆ null                ┆ true                │
│ 2025-06-06 08:25:12.837 ┆ 1990484925050034825 ┆ 376089619872553315  ┆ true                │
│ 2025-06-06 08:37:10.989 ┆ 1063926452435308816 ┆ 1990484925050034825 ┆ true                │
│ 2025-06-06 08:38:23.088 ┆ 5843758373603936520 ┆ 1063926452435308816 ┆ true                │
└─────────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┘

The SQL equivalent will depend on the query engine - Trino uses `$` as the metadata table identifier

In [4]:
history = pl.read_database(
    'SELECT * FROM housing."staging_prices$history" order by made_current_at', engine
)
with pl.Config(thousands_separator=None):
    display(history)

shape: (4, 4)
┌─────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ made_current_at             ┆ snapshot_id         ┆ parent_id           ┆ is_current_ancestor │
│ ---                         ┆ ---                 ┆ ---                 ┆ ---                 │
│ datetime[μs, UTC]           ┆ i64                 ┆ i64                 ┆ bool                │
╞═════════════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╡
│ 2025-06-06 08:18:03.013 UTC ┆ 376089619872553315  ┆ null                ┆ true                │
│ 2025-06-06 08:25:12.837 UTC ┆ 1990484925050034825 ┆ 376089619872553315  ┆ true                │
│ 2025-06-06 08:37:10.989 UTC ┆ 1063926452435308816 ┆ 1990484925050034825 ┆ true                │
│ 2025-06-06 08:38:23.088 UTC ┆ 5843758373603936520 ┆ 1063926452435308816 ┆ true                │
└─────────────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┘

Now that we have a list of snapshots, we can demonstrate timetravelling. We loaded 2024, 2023 and 2022 data into our table, so we should see different counts in each snapshot

In [5]:
pl.read_database("SELECT count(transaction_id) as num_rows FROM housing.staging_prices", engine)

shape: (1, 1)
┌───────────┐
│ num_rows  │
│ ---       │
│ i64       │
╞═══════════╡
│ 2,684,736 │
└───────────┘

The time travel syntax also varies by query engine, but Trino uses the `FOR VERSION AS OF` syntax

In [7]:
pl.read_database(
    "SELECT count(transaction_id) as num_rows from housing.staging_prices for version as of 1063926452435308816",
    engine,
)

shape: (1, 1)
┌───────────┐
│ num_rows  │
│ ---       │
│ i64       │
╞═══════════╡
│ 2,684,736 │
└───────────┘

Pyiceberg exposes a similar API, where we can specify the `snapshot_id` we want to read

In [9]:
house_prices_t.scan(
    snapshot_id=1063926452435308816, selected_fields=["transaction_id"]
).to_arrow().num_rows

2684736

Since most libriaries build on Pyiceberg, you'll see similar APIs there

In [10]:
pl.scan_iceberg(house_prices_t, snapshot_id=1063926452435308816).select(
    pl.count("transaction_id")
).collect()

shape: (1, 1)
┌────────────────┐
│ transaction_id │
│ ---            │
│ u32            │
╞════════════════╡
│ 2,684,736      │
└────────────────┘

SQL offers us some niceties here in that we can timetravel via timestamps as well, and Trino will do the work of looking up the snapshot closest in time

In [12]:
pl.read_database(
    "SELECT count(transaction_id) as num_rows from housing.staging_prices for timestamp as of timestamp '2025-06-06 08:40:00'",
    engine,
)

shape: (1, 1)
┌───────────┐
│ num_rows  │
│ ---       │
│ i64       │
╞═══════════╡
│ 2,684,736 │
└───────────┘

Remembering these snapshot ids or pinpointing the exact time we're interested in is tricky for our human brains, so Iceberg supports tagging so that we can provide human-readable references to a given snapshot.

In [13]:
house_prices_t.manage_snapshots().create_tag(
    1063926452435308816, "initial commit"
).commit()

In [14]:
with pl.Config(thousands_separator=None):
    display(pl.from_arrow(house_prices_t.inspect.refs()))

shape: (2, 6)
┌────────────────┬────────┬──────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ name           ┆ type   ┆ snapshot_id      ┆ max_reference_a ┆ min_snapshots_t ┆ max_snapshot_ag │
│ ---            ┆ ---    ┆ ---              ┆ ge_in_ms        ┆ o_keep          ┆ e_in_ms         │
│ str            ┆ cat    ┆ i64              ┆ ---             ┆ ---             ┆ ---             │
│                ┆        ┆                  ┆ i64             ┆ i32             ┆ i64             │
╞════════════════╪════════╪══════════════════╪═════════════════╪═════════════════╪═════════════════╡
│ initial commit ┆ TAG    ┆ 1063926452435308 ┆ null            ┆ null            ┆ null            │
│                ┆        ┆ 816              ┆                 ┆                 ┆                 │
│ main           ┆ BRANCH ┆ 5843758373603936 ┆ null            ┆ null            ┆ null            │
│                ┆        ┆ 520              ┆                 ┆                 ┆                 │
└────────────────┴────────┴──────────────────┴─────────────────┴─────────────────┴─────────────────┘

Now that we have this tag, we can reference it directly in our SQL statement

In [15]:
pl.read_database(
    "SELECT count(transaction_id) as num_rows from housing.staging_prices for version as of 'initial commit'",
    engine,
)

shape: (1, 1)
┌───────────┐
│ num_rows  │
│ ---       │
│ i64       │
╞═══════════╡
│ 2,684,736 │
└───────────┘

Pyiceberg is a bit more clunky - since we need to pass a snapshot ID, we need to use Pyiceberg to lookup the snapshot_id for our tag

In [16]:
pl.scan_iceberg(
    house_prices_t,
    snapshot_id=house_prices_t.snapshot_by_name("initial commit").snapshot_id,
).select(pl.count("transaction_id")).collect()

shape: (1, 1)
┌────────────────┐
│ transaction_id │
│ ---            │
│ u32            │
╞════════════════╡
│ 2,684,736      │
└────────────────┘

We can permanently rollback a change, though this is not available through Pyiceberg

In [17]:
with engine.connect() as conn:
    conn.execute(
        sa.text(
            "ALTER TABLE housing.staging_prices EXECUTE rollback_to_snapshot(1063926452435308816)"
        )
    ).fetchone()

```{warning}
The current schema of the table remains unchanged even if we rollback. Current schema is set to include the `_loaded_at` column we added earlier
```

In [18]:
pl.read_database("SELECT count('transaction_id') as num_rows from housing.staging_prices", engine)

shape: (1, 1)
┌───────────┐
│ num_rows  │
│ ---       │
│ i64       │
╞═══════════╡
│ 2,684,736 │
└───────────┘

When making metadata changes in a different query engine it's important to refresh our Pyiceberg metadata, since metadata is cached

In [19]:
house_prices_t.refresh();

In [20]:
pl.scan_iceberg(house_prices_t).select(pl.col("transaction_id").len().alias('num_rows')).collect()

shape: (1, 1)
┌───────────┐
│ num_rows  │
│ ---       │
│ u32       │
╞═══════════╡
│ 2,684,736 │
└───────────┘

In [21]:
with pl.Config(thousands_separator=None):
    display(pl.from_arrow(house_prices_t.inspect.history()))

shape: (5, 4)
┌─────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ made_current_at         ┆ snapshot_id         ┆ parent_id           ┆ is_current_ancestor │
│ ---                     ┆ ---                 ┆ ---                 ┆ ---                 │
│ datetime[ms]            ┆ i64                 ┆ i64                 ┆ bool                │
╞═════════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╡
│ 2025-06-06 08:18:03.013 ┆ 376089619872553315  ┆ null                ┆ true                │
│ 2025-06-06 08:25:12.837 ┆ 1990484925050034825 ┆ 376089619872553315  ┆ true                │
│ 2025-06-06 08:37:10.989 ┆ 1063926452435308816 ┆ 1990484925050034825 ┆ true                │
│ 2025-06-06 08:38:23.088 ┆ 5843758373603936520 ┆ 1063926452435308816 ┆ false               │
│ 2025-06-06 08:45:47.966 ┆ 1063926452435308816 ┆ 1990484925050034825 ┆ true                │
└─────────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┘

## Cleaning up

Iceberg provides various routines to clean up files and metadata as orphan files and unused data pile up. Depending on your catalogue, this may be an automated process, but we can manually trigger them via Trino

In [22]:
with engine.connect() as conn:
    # Remove snapshots and corresponding metadata
    conn.execute(
        sa.text(
            "ALTER TABLE housing.staging_prices EXECUTE expire_snapshots(retention_threshold => '0d')"
        )
    ).fetchone()
    # Remove orphaned files not referenced by metadata
    conn.execute(
        sa.text(
            "ALTER table housing.staging_prices execute remove_orphan_files(retention_threshold => '0d')"
        )
    ).fetchone()
    # Co-locate manifests based on partitioning
    conn.execute(
        sa.text("ALTER TABLE housing.staging_prices EXECUTE optimize_manifests")
    ).fetchone()
    # Compact small files into larger
    conn.execute(
        sa.text("ALTER table housing.staging_prices execute optimize")
    ).fetchone()

In [23]:
with pl.Config(thousands_separator=None):
    display(pl.read_database(
        'SELECT * FROM housing."staging_prices$history" order by made_current_at', engine
    ))

shape: (3, 4)
┌─────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┐
│ made_current_at             ┆ snapshot_id         ┆ parent_id           ┆ is_current_ancestor │
│ ---                         ┆ ---                 ┆ ---                 ┆ ---                 │
│ datetime[μs, UTC]           ┆ i64                 ┆ i64                 ┆ bool                │
╞═════════════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╡
│ 2025-06-06 08:37:10.989 UTC ┆ 1063926452435308816 ┆ 1990484925050034825 ┆ true                │
│ 2025-06-06 08:47:14.425 UTC ┆ 7679618584076361159 ┆ 1063926452435308816 ┆ true                │
│ 2025-06-06 08:47:18.659 UTC ┆ 6938640996140855726 ┆ 7679618584076361159 ┆ true                │
└─────────────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┘